In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from scipy import interp
import matplotlib.pyplot as plt
import time

In [5]:
#loading data
dataset = np.load('/content/drive/My Drive/data.row/ASD2.npz') #Dataset ready in numpy array (removing background, resizing, and transforming into grayscale)
X = dataset['X']
y = dataset['y']

print(X.shape)
print(y.shape)

(3103, 224, 224, 3)
(3103, 1)


In [6]:
#Hyperparameters
nfolds = 3
nEpochs = 30
nBatch= 16
#inputDim = X.shape[1] # Count of features

In [7]:
kfold = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=1).split(X, y) #Cross validation
for train, test in kfold:
  print(X[train].shape, X[test].shape ,y[train].shape, y[test].shape )

(2068, 224, 224, 3) (1035, 224, 224, 3) (2068, 1) (1035, 1)
(2069, 224, 224, 3) (1034, 224, 224, 3) (2069, 1) (1034, 1)
(2069, 224, 224, 3) (1034, 224, 224, 3) (2069, 1) (1034, 1)


In [8]:
resnet50 = tf.keras.applications.ResNet50() 
#resnet50.summary()

102973440/102967424 [==============================] - 3s 0us/step


In [9]:
x = resnet50.layers[-2].output

In [10]:
output = Dense(units=1, activation='sigmoid')(x)

In [11]:
model = Model(inputs=resnet50.input, outputs=output)

In [12]:
#for layer in model.layers[:-23]:
#   layer.trainable = False

for layer in model.layers[175:]:
    layer.trainable = True
for layer in model.layers[:175]:
    layer.trainable = False
#for i, layer in enumerate(model.layers):
    #print(i, layer.name, layer.trainable)

In [13]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()

In [15]:
tprs = []
aucs = []

recall =[]
precision = []

mean_fpr = np.linspace(0, 1, 100)

start = time.time()
hists_CNN = []

In [20]:
#hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4]}]
# set up GridSearchCV()
#model_cv = sklearn.model_selection.GridSearchCV(estimator = model, 
                      #  param_grid = {}, 
                     #   scoring= 'accuracy', 
                     #   cv = kfold, 
                      #  verbose = 2,
                      #  return_train_score=True)

In [ ]:
#این قسمت رو از کگل برای انجام کافولد پیدا کردم ولی هنوز نتونستم باهاش به نتیجه برسم.البته که یکم کدش برام پیچیده هست.
#from tensorflow.keras.applications import DenseNet201

#def get_model():
    
    with strategy.scope():
        rnet = DenseNet201(
            input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
            weights='imagenet',
            include_top=False
        )

        # trainable rnet
        rnet.trainable = True

        model = tf.keras.Sequential([
            rnet,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(len(CLASSES), activation='softmax')
        ])

    model.compile(
        optimizer='adam',
        loss = 'sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy']
    )

    return model

def train_cross_validate(folds = 4):
    histories = []
    models = []
    #early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 4)
    #kfold = KFold(folds, shuffle = True, random_state = 42)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
        train_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[trn_ind]['TRAINING_FILENAMES']), labeled = True)
        val_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[val_ind]['TRAINING_FILENAMES']), labeled = True, ordered = True)
        checkpoint_name = f'model_fold_{fold + 1}' + '.h5'
        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_name, save_best_only = True, save_weights_only = True)
        model = get_model()
        history = model.fit(
        get_training_dataset(train_dataset), 
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs = EPOCHS,
        callbacks = [lr_callback, early_stopping, model_checkpoint],
        validation_data = get_validation_dataset(val_dataset)
        )
        print('Load best weights for model prediction')
        model.load_weights(checkpoint_name)
        models.append(model)
        histories.append(history)
        
    return histories, models

def train_and_predict(folds = 4):
    test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.
    test_images_ds = test_ds.map(lambda image, idnum: image)
    print('Start training {} models'.format(folds))
    histories, models = train_cross_validate(folds = folds)
    print('Computing predictions...')
    # get the mean probability of the folds models
    probabilities = np.average([models[i].predict(test_images_ds) for i in range(folds)], axis = 0)
    predictions = np.argmax(probabilities, axis=-1)
    print('Generating submission.csv file...')
    test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
    test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
    np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
    return histories, models
    
# run train and predict
histories, models = train_and_predict(folds = 4)

In [25]:
#Fiting the model 

hist =model.fit(X[train], y[train], validation_split=0.2, epochs=nEpochs, batch_size=nBatch, verbose=2)
  #pred = model.predict(X[test]).ravel()

  #Calculating Recall and precision
  #recall.append( recall_score(y[test], pred.round()) )
  #precision.append( precision_score(y[test], pred.round()) )

  #ROC AUC
  #fpr, tpr, thresholds = roc_curve(y[test], pred)
  #tprs.append(interp(mean_fpr, fpr, tpr))
  #tprs[-1][0] = 0.0
  #roc_auc = auc(fpr, tpr)
  #print(roc_auc)
  #aucs.append(roc_auc)
  #hists_CNN.append(hist)


#print("Avg AUC:", np.mean(aucs))

#print("Avg Recall:", np.mean(recall))
#print("Avg Precision:", np.mean(precision))



Epoch 1/30
104/104 - 6s - loss: 0.6893 - accuracy: 0.6816 - val_loss: 1.2064 - val_accuracy: 0.0000e+00
Epoch 2/30
104/104 - 4s - loss: 0.6456 - accuracy: 0.6985 - val_loss: 2.2023 - val_accuracy: 0.0000e+00
Epoch 3/30
104/104 - 4s - loss: 0.6630 - accuracy: 0.6912 - val_loss: 1.0183 - val_accuracy: 0.0000e+00
Epoch 4/30
104/104 - 4s - loss: 0.6439 - accuracy: 0.6985 - val_loss: 0.6381 - val_accuracy: 0.9952
Epoch 5/30
104/104 - 4s - loss: 0.6631 - accuracy: 0.6659 - val_loss: 1.9981 - val_accuracy: 0.0000e+00
Epoch 6/30
104/104 - 4s - loss: 0.6836 - accuracy: 0.6931 - val_loss: 1.1282 - val_accuracy: 0.0000e+00
Epoch 7/30
104/104 - 4s - loss: 0.7145 - accuracy: 0.6453 - val_loss: 0.4007 - val_accuracy: 1.0000
Epoch 8/30
104/104 - 4s - loss: 0.6652 - accuracy: 0.6737 - val_loss: 1.8835 - val_accuracy: 0.0000e+00
Epoch 9/30
104/104 - 4s - loss: 0.6461 - accuracy: 0.6858 - val_loss: 1.0731 - val_accuracy: 0.0000e+00
Epoch 10/30
104/104 - 4s - loss: 0.6242 - accuracy: 0.7208 - val_loss: 2

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)